In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [82]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
# Import necessary modules
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import nltk
import spacy
!spacy download en_core_web_sm
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import SyllableTokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from spacy.lang.en.stop_words import STOP_WORDS

2023-07-17 03:49:24.881057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Data Cleaning and Analysis Functions

In [74]:
# Scrape the data
def scrape_data(url):
  html_text = requests.get(url).text
  soup = BeautifulSoup(html_text, 'lxml')

  if soup:
    titles = soup.find('h1', class_ = ['post-title', 'tdb-title-text'])

  if titles == None:
    return None
  # Get all the titles in the article
  for title in titles:
      title_name = title.text

  trying = soup.find_all('div', class_ = ['post-content', 'tdb-block-inner td-fix-index'])
  new_page = []

  # Get all the content using the 'p' tag because the content of the articleis stored in 'p' tags
  for trial in trying:
      # In each 'div' tag if there is no 'p' tag, move on to the next 'div' tag
      if trial.find_all('p') == []:
          continue
      else:
          # If there is a 'p' tag, find them  all and save them in the variable 'new page'
          new_page = trial.find_all('p')

  # The entire article is not saved in just 1 'p' tag but several
  for i in range(len(new_page)):

    # for each 'p' tag, save only the text
    new_page[i] = new_page[i].text

  page = ' '
  # Convert the list into a single string
  page = page.join(new_page)

  # Join the article and the title together
  article = title_name + " " + page

  return article, title_name

In [31]:
# Function to remove stopwords
def remove_stopwords(some_text):

  # Remove hyperlinks
  some_text = re.sub(r'http\S+|www\S+|https\S+', '', some_text)

  # Remove HTML tags
  some_text = BeautifulSoup(some_text, 'html.parser').get_text()

  # Remove anything that is not a word or white space"
  # some_text = re.sub(r'[^\w\s]', '', some_text)

  # Remove numbers from the text
  some_text = re.sub(r'[\d+]', '', some_text)

  #Tokenize
  doc = word_tokenize(some_text)
  lemmatizer = WordNetLemmatizer()
  lemmas = [lemmatizer.lemmatize(token) for token in doc]
  # Remove stopwords and non-alphabetic tokens
  a_lemmas = [lemma for lemma in lemmas if lemma.isalpha() and lemma not in STOP_WORDS]
  res = ' '. join(a_lemmas)

  return res

In [32]:
# Function to calculate positivity score
def calculate_positivity_score(some_text):
  with open('/content/drive/MyDrive/Punch Project/Dictionary/positive-words.txt', 'r', encoding='latin1') as text_file:
    text = text_file.read()

    # Tokenize the text
    doc_pos = word_tokenize(text)

    # Remove words in the positive words list that are also in the stopwords list
    pos_list = [word for word in doc_pos if word not in STOP_WORDS]

  # Tokenize the text
  doc = word_tokenize(some_text)

  # Calculate the postitivity score
  positive_score = 0
  for word in doc:
    if word in pos_list:
      positive_score += 1

  return positive_score

In [33]:
# Function to calculate negativity score
def calculate_negativity_score(some_text):
  with open('/content/drive/MyDrive/Punch Project/Dictionary/negative-words.txt', 'r', encoding='latin1') as text_file:
    text = text_file.read()

    # Tokenize the text
    doc_neg = word_tokenize(text)

    # Remove words in the negative words list that are also in the stopwords list
    neg_list = [word for word in doc_neg if word not in STOP_WORDS]

  # Tokenize the text
  doc = word_tokenize(some_text)

  # Calculate the negativity score
  negative_score = 0
  for word in doc:
    if word in neg_list:
      negative_score += 1

  return negative_score

In [34]:
# Function to calculate subjectivity score
def calculate_polarity_score(positive_score, negative_score):
  polarity_score = (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)

  return polarity_score

In [35]:
# Function to calculate word count
def calculate_word_count(some_text):
  doc = word_tokenize(some_text)
  word_count = len(doc)

  return word_count

In [36]:
# Function to calculate subjectivity score
def calculate_subjectivity_score(some_text, positive_score, negative_score):
  word_count = calculate_word_count(some_text)

  subjectivity_score = (positive_score + negative_score)/(word_count + 0.000001)

  return subjectivity_score

In [37]:
# Function to calculate average number of words per sentence
def calculate_avg_sentence_len(some_text, article):

  # Divide the article into a list of sentences
  no_sentence = sent_tokenize(article)

  word_count = calculate_word_count(some_text)

  # Average sentence length
  avg_sentence_len = word_count/len(no_sentence)

  return avg_sentence_len

In [38]:
# Function to calculate complex word count
def calculate_complex_word(some_text):
  # Instantiate the syllable tokenizer
  SSP = SyllableTokenizer()

  # A list of words with more than 2 syllables in our text
  complex_list = [token for token in word_tokenize(some_text) if len(SSP.tokenize(token)) > 2]

  no_complex_words = len(complex_list)
  return no_complex_words

In [39]:
# Function to calculate percentage of complex words
def calculate_per_complex_words(no_complex_words, no_words):
  # Percentage of complex words
  per_complex_words = no_complex_words/no_words

  return per_complex_words

In [40]:
# Function to calculate Fog Index
def calculate_fog_index(some_text, avg_sentence_len, no_complex_words):

  no_words = calculate_word_count(some_text)

  # Percentage of complex words
  per_complex_words = calculate_per_complex_words(no_complex_words, no_words)

  # Calculate fog index
  fog_index = 0.4 * (avg_sentence_len + per_complex_words)

  return fog_index

In [41]:
# Function to calculate syllable count per word
def calculate_no_syllable_per_word(some_text):
  # Instantiate our syllable Tokenizer
  SSP = SyllableTokenizer()

  # Load your Spacy tokenizer
  nlp = spacy.load("en_core_web_sm")

  some_text_token = nlp(some_text)

  # Remove all the "es" and "ed" syllables using lemmatization
  lemmas = [token.lemma_ for token in some_text_token]

  # Create a list of the syllables for each word
  syllable_list = [SSP.tokenize(token) for token in lemmas]

  no_syllable = 0
  for word in syllable_list:
    # Calculate the number of syllables for each word and add them up
    no_syllable += len(word)

  token_list = [token.text for token in some_text_token]

  syllable_per_word = no_syllable/len(token_list)

  return syllable_per_word

In [42]:
# Function to extract personal pronouns
def calculate_personal_pron(some_text):

  # Find the following words: I, we, my, ours. re.I ensures that you search for those words case INsensitive while (?-i:us) ensures that us is case sensitive
  pronounRegex = re.compile(r'(I|we|my|ours|(?-i:us))',re.I)

  pronouns = pronounRegex.findall(some_text)

  personal_pron_count = len(pronouns)

  return personal_pron_count

In [43]:
# Function to calculate average word length
def calculate_avg_word_len(some_text, word_count):
  total_char = len(re.sub('\s+', "", some_text))

  avg_word_len = total_char/word_count

  return avg_word_len

## Putting it all together

In [84]:
input_file = '/content/drive/MyDrive/Punch Project/The Punch.xlsx'
output_file = '/content/drive/MyDrive/Punch Project/The Punch Output File.xlsx'
output_file_1 = '/content/drive/MyDrive/Punch Project/The Punch Cleaned File.xlsx'

# Select the columns in the output excel file
df = pd.read_excel(output_file)
df_1 = pd.read_excel(output_file_1)
columns = list(df.columns)
columns_1 = list(df_1.columns)

# Create an empty DataFrame for the output
output_df = pd.DataFrame(columns=columns)
output_df_cleaned = pd.DataFrame(columns = columns_1)


# Iterate through each row of the input DataFrame
for index, row in df.iterrows():
  url = row['URL']

  print(url)

  article, title_name = scrape_data(url)  # Scrape data from the URL

  if article:
    data = remove_stopwords(article) # Remove stopwords from the list

    # A dataframe containing the URL, cleaned data and the tag
    output_df_cleaned.loc[index] = [df_1['URL'][index], title_name, data, df_1['TAGS'][index]]

    positive_score = calculate_positivity_score(data) # Calculate the positive score
    negative_score = calculate_negativity_score(data) # Calculate the negative score
    polarity_score = calculate_polarity_score(positive_score, negative_score) # Calculate the polarity score
    word_count = calculate_word_count(data) # Calculate the word count
    subjectivity_score = calculate_subjectivity_score(data, positive_score, negative_score) # Calculate the subjectivity score
    avg_sentence_len = calculate_avg_sentence_len(data, article) # Calculate the average number of words per sentence
    no_complex_words = calculate_complex_word(data) # Calculate the number of complex words
    per_complex_words = calculate_per_complex_words(no_complex_words, word_count) # Calculate the percentage of complex words
    fog_index = calculate_fog_index(data, avg_sentence_len, no_complex_words) # Calculate the fog index
    syllable_per_word = calculate_no_syllable_per_word(data) # Calculate the syllable per word
    personal_pron_count = calculate_personal_pron(data) # Calculate the number of personal pronouns
    avg_word_len = calculate_avg_word_len(data, word_count) # Calculate the average word length

    output_df.loc[index] = [df['DATE PUBLISHED'][index], df['URL'][index], title_name, df['TAGS'][index], positive_score, negative_score, polarity_score, subjectivity_score,
                            avg_sentence_len, per_complex_words, fog_index, no_complex_words, word_count,
                            syllable_per_word, personal_pron_count, avg_word_len]
  else:
    output_df.loc[index] = [df['DATE PUBLISHED'][index], df['URL'][index], title_name, df['TAGS'][index], None, None, None, None,
                            None, None, None, None, None, None, None, None]

    output_df_cleaned.loc[index] = [df_1['URL'][index], None, df_1['TAGS'][index]]

    continue

# Save the output DataFrame to an Excel file
output_file = '/content/drive/MyDrive/Punch Project/The Punch Output File.xlsx'
output_df.to_excel(output_file, index = False)

output_file_1 = '/content/drive/MyDrive/Punch Project/The Punch Cleaned File.xlsx'
output_df_cleaned.to_excel(output_file_1, index = False)

https://punchng.com/presidential-inauguration-obi-didnt-call-for-boycott-postponement-lp/
https://punchng.com/lets-reclaim-pdp-lost-glory-atiku-tasks-party-members/
https://punchng.com/tinubull-be-fair-to-all-ex-lawmaker/
https://punchng.com/just-in-atiku-obaseki-attend-pdps-reception-for-new-returning-governors/
https://punchng.com/im-the-best-candidate-for-senate-president-osita-izunaso-insists/
https://punchng.com/lp-faction-wants-tinubu-sworn-in-says-obi-asking-for-impossible/
https://punchng.com/why-fg-approved-contracts-18-days-to-handover-minister/
https://punchng.com/tinubu-celebrates-with-fasoranti-at-97/
https://punchng.com/10th-assembly-wase-betara-other-aspirants-protest-zoning/
https://punchng.com/oyo-pdp-mourns-adagunodo/
https://punchng.com/obi-lp-back-atiku-on-request-for-live-broadcast/
https://punchng.com/akeredolu-knocks-apc-over-nass-leadership-zoning-formula/
https://punchng.com/adagunodo-nigeria-lost-a-patriot-atiku-mourns/
https://punchng.com/wase-betara-other-as

/usr/local/lib/python3.10/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'í'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'á'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'ú'
  warnings.warn(


https://punchng.com/adeleke-four-other-governors-court-handed-victories/
https://punchng.com/house-speakership-apc-picked-me-based-on-competence-says-abbas/
https://punchng.com/ekiti-senators-others-behind-akpabio-oyebanji/
https://punchng.com/breaking-app-withdraws-petition-against-tinubu/
https://punchng.com/senate-presidency-north-central-lawmakers-reject-apc-consensus-list/
https://punchng.com/accept-adelekes-victory-oyetola-tells-osun-apc/
https://punchng.com/presidential-poll-tribunal-to-consider-pdp-lps-live-broadcast-request/
https://punchng.com/speaker-five-aggrieved-aspirants-team-up-against-abbas/
https://punchng.com/final-victory-buhari-tinubu-congratulate-adeleke-supporters-jubilate/
https://punchng.com/bvas-will-dash-hope-of-dishonest-politicians-atiku/
https://punchng.com/osun-verdict-buhari-urges-support-for-adeleke-led-govt/
https://punchng.com/abia-labour-party-lawmaker-backs-abass-as-hor-speaker/
https://punchng.com/presidential-poll-tribunal-shouldnt-be-intimidated-

/usr/local/lib/python3.10/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'é'
  warnings.warn(


https://punchng.com/obi-visits-soyinka-one-month-after-clash-with-supporters/
https://punchng.com/sylva-sole-financier-of-bayelsa-apc-party-chair/
https://punchng.com/senate-presidency-south-east-senators-reject-zoning-to-south-south-caution-tinubu/


...

https://healthwise.punchng.com/insufficient-oncologists-equipment-affecting-cancer-treatment-says-ncs/
https://healthwise.punchng.com/political-will-required-to-immunise-67-million-children-missed-during-covid-19-pandemic-unicef/
https://healthwise.punchng.com/my-perseverance-helped-me-to-overcome-all-odds-physically-challenged-graduate/
https://healthwise.punchng.com/mdcn-wrong-to-downgrade-doctors-certificates-nard-president/


/usr/local/lib/python3.10/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'é'
  warnings.warn(


https://healthwise.punchng.com/nurses-lament-bayelsas-refusal-to-pay-new-hazard-allowance/
https://healthwise.punchng.com/deadly-anthrax-disease-outbreak-looms-over-poor-awareness/
https://healthwise.punchng.com/why-nigeria-was-excluded-from-malaria-vaccine-allocation-says-gavi/
https://healthwise.punchng.com/inhaling-nitrous-oxide-to-get-high-may-cause-brain-damage-mental-illnesses-psychiatrists/
https://healthwise.punchng.com/rising-antenatal-care-at-tbas-fuels-child-hiv-cases-gynaecologists/
https://healthwise.punchng.com/why-sweating-excessively-at-night-should-raise-serious-health-concerns-physicians/
https://healthwise.punchng.com/world-not-on-track-to-achieve-universal-access-to-water-sanitation-un-agencies/


/usr/local/lib/python3.10/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'ã'
  warnings.warn(


https://healthwise.punchng.com/exercising-without-sufficient-sleep-may-cause-cognitive-decline-study/
https://healthwise.punchng.com/call-off-strike-to-save-lives-nasarawa-govt-begs-striking-doctors/
https://healthwise.punchng.com/sanwo-olu-restates-commitment-to-safe-water/
https://healthwise.punchng.com/public-health-expert-tells-fg-to-allocate-70-healthcare-budget-to-phcs/
https://healthwise.punchng.com/working-beyond-40-hours-weekly-may-cause-occupational-hazards-public-health-expert-warns/
https://healthwise.punchng.com/excessive-alcohol-intake-overburdens-liver-reduces-libido-nutritionist-warns/
https://healthwise.punchng.com/nigeria-records-798-diphtheria-cases-80-deaths-in-33-lgas/
https://healthwise.punchng.com/lagos-needs-efficient-regulatory-framework-for-hygiene-promotion-sanwo-olu/
https://healthwise.punchng.com/lessons-from-covid-19-should-stimulate-fg-stakeholders-to-fund-niprd-dg/
https://healthwise.punchng.com/nutrition-experts-task-govt-on-food-fortification-enforceme

## REFERENCES
The negative and positive text files were sourced from:
https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#lexicon

Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews."
;       Proceedings of the ACM SIGKDD International Conference on Knowledge
;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle,
;       Washington, USA,
